In [1]:
import os, glob, librosa, pickle
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Dense

from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
with open ('data_2_chroma_stft', 'rb') as fp:
    dp = pickle.load(fp)

In [3]:
with open ('label_2_chroma_stft', 'rb') as fp:
    lb = pickle.load(fp)

In [4]:
len(dp)

43584

In [5]:
len(dp[0])

4068

In [6]:
onehot_encoder = OneHotEncoder(sparse=False)

In [7]:
lb = np.array(lb)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(dp,lb,test_size = 0.5, shuffle = True, random_state = 42, stratify = lb)

In [9]:
lbl = y_train.reshape((len(y_train),1))

In [10]:
onehot_encoded = onehot_encoder.fit_transform(lbl)

C:\Users\Gurpreet\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [11]:
onehot_encoded.shape

(21792, 109)

In [12]:
X_train = np.array(X_train)

In [13]:
X_train.shape

(21792, 4068)

In [14]:
scaler = preprocessing.StandardScaler().fit(X_train)

In [15]:
X_train = scaler.transform(X_train)

In [16]:
X_train.shape

(21792, 4068)

In [18]:
X_train = X_train.reshape((21792,4068,1))

In [19]:
X_train.shape

(21792, 4068, 1)

In [20]:
dnn = Sequential()

In [21]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, BatchNormalization

In [22]:
dnn.add(Conv1D(filters = 16, kernel_size=(10), strides=(1), activation='relu', input_shape=(4068,1)))
dnn.add(MaxPooling1D(pool_size=(5), strides = (2)))
dnn.add(BatchNormalization())

dnn.add(Conv1D(filters = 32, kernel_size=(10), activation='relu'))
dnn.add(MaxPooling1D(pool_size=(5), strides = (2)))
dnn.add(BatchNormalization())

dnn.add(Conv1D(filters = 64, kernel_size=(10), activation='relu'))
dnn.add(MaxPooling1D(pool_size=(5), strides = (1)))
dnn.add(BatchNormalization())

dnn.add(Flatten())
dnn.add(Dense(300, activation='relu'))

dnn.add(Dense(109, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [23]:
dnn.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

In [24]:
dnn.fit(X_train, onehot_encoded, epochs = 2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
21792/21792 [==============================] - 627s 29ms/step - loss: 2.7214 - acc: 0.3508
Epoch 2/2
21792/21792 [==============================] - 620s 28ms/step - loss: 0.8247 - acc: 0.7689


In [25]:
X_test = scaler.transform(X_test)

In [26]:
X_test.shape

(21792, 4068)

In [27]:
X_test = X_test.reshape((21792,4068,1))

In [28]:
lbl_test = y_test.reshape((len(y_test),1))

In [29]:
onehot_encoded_test = onehot_encoder.fit_transform(lbl_test)

C:\Users\Gurpreet\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [30]:
pred = dnn.predict(X_test)

In [31]:
pred.shape

(21792, 109)

In [32]:
from sklearn.metrics import accuracy_score

In [33]:
lb_test = []
for i in range(len(pred)):
    lb_test.append(np.argmax(pred[i]))

In [34]:
lb_test = np.array(lb_test)

In [35]:
lb_test.shape

(21792,)

In [36]:
y_test.shape

(21792,)

In [37]:
accuracy_score(y_test, lb_test)

0.523724302496329

In [38]:
from sklearn.metrics import matthews_corrcoef

In [39]:
matthews_corrcoef(y_test, lb_test)

0.5194676498290198

In [40]:
from keras.models import load_model

In [41]:
dnn.save('chroma_stft_cnn_1.h5')

In [42]:
clf = load_model('chroma_stft_cnn_1.h5')

In [43]:
prd = clf.predict(X_test)

In [44]:
tr = []
for i in range(len(prd)):
    tr.append(np.argmax(prd[i]))

In [45]:
tr = np.array(tr)

In [46]:
tr.shape

(21792,)

In [47]:
accuracy_score(y_test, tr)

0.523724302496329

In [48]:
matthews_corrcoef(y_test, tr)

0.5194676498290198

In [49]:
from sklearn.externals import joblib

In [50]:
joblib.dump(scaler, 'scaler_chroma_stft_cnn.save')

['scaler_chroma_stft_cnn.save']

In [51]:
sclr = joblib.load('scaler_cnn.save')

In [71]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [72]:
from sklearn.metrics import confusion_matrix

In [73]:
cf = confusion_matrix(y_test, tr)

In [74]:
plt.figure(figsize = (218,218))
sn.heatmap(cf, annot=True)

In [2]:
with open ('data_2_chroma_stft', 'rb') as fp:
    dp = pickle.load(fp)

with open ('label_2_chroma_stft', 'rb') as fp:
    lb = pickle.load(fp)
    
X_train, X_test, y_train, y_test = train_test_split(dp,lb,test_size = 0.5, shuffle = True, random_state = 42, stratify = lb)

In [3]:
from sklearn.externals import joblib
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import precision_recall_fscore_support


sclr = joblib.load('scaler_chroma_stft_cnn.save')
clf = load_model('chroma_stft_cnn_1.h5')
X_test = sclr.transform(X_test)
X_test = X_test.reshape((21792,4068,1))

pred = clf.predict(X_test)


lb_test = []
for i in range(len(pred)):
    lb_test.append(np.argmax(pred[i]))

lb_test = np.array(lb_test)

acc = accuracy_score(y_test, lb_test)
print("Accuracy : ",acc)

mcc = matthews_corrcoef(y_test, lb_test)
print("MCC : ",mcc)

prf = precision_recall_fscore_support(y_test, lb_test, average='macro')
print("Precission : ", prf[0]," Recall : ", prf[1]," F1 Score : ",prf[2])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Accuracy :  0.523724302496329
MCC :  0.5194676498290198
Precission :  0.5429106414485422  Recall :  0.519460195766227  F1 Score :  0.5187270866692633
